Необходимые импорты

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score
import seaborn as sns

Загрузка данных

In [2]:
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names
target_names = data.target_names

print(f"Размер датасета: {X.shape}")
print(f"Количество признаков: {len(feature_names)}")
print(f"Целевые классы: {target_names}")
print(f"Соотношение классов: {np.bincount(y)}")


Размер датасета: (569, 30)
Количество признаков: 30
Целевые классы: ['malignant' 'benign']
Соотношение классов: [212 357]


Разделение на тренировочную и тестовую выборку


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain size: {X_train.shape}, Test size: {X_test.shape}")



Train size: (455, 30), Test size: (114, 30)


In [4]:
from boosting_classifier import GradientBoostingClassifier as gb
from sklearn.ensemble import GradientBoostingClassifier as gbsk

модели

Обучение

In [5]:
gb = gb(n_estimators=100, max_depth=5)
gbsk = gbsk()
gb.fit(X_train, y_train)
gbsk.fit(X_train, y_train)

GradientBoostingClassifier()

предсказания

Используем реколл, так как не хотим пропустить больных при обследовании


In [7]:
y_pred = gb.predict(X_test)
y_pred_proba = gb.predict_proba(X_test)

y_pred_sk = gbsk.predict(X_test)
y_pred_proba_sk = gbsk.predict_proba(X_test)

In [8]:
print("Реколл для больных моей реализации", recall_score(y_test, y_pred, pos_label=0))
print("Реколл для больных склерн реализации", recall_score(y_test, y_pred_sk, pos_label=0))


Реколл для больных моей реализации 0.9285714285714286
Реколл для больных склерн реализации 0.9047619047619048


Нормализуем данные

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

С помощью кросс валидации подбираем гиперпараметры


In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Создание пайплайна для градиентного бустинга
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingClassifier(random_state=42))
])

# Параметры для GridSearchCV
param_grid = {
    'gb__n_estimators': [100, 200],
    'gb__max_depth': [3, 5, 7],
    'gb__min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

print("\nПоиск параметров для градиентного бустинга...")
grid_search.fit(X_train, y_train)

print(f"\nЛучшие параметры: {grid_search.best_params_}")
print(f"Лучший recall (CV): {grid_search.best_score_:.4f}")

best_gb_model = grid_search.best_estimator_

from sklearn.metrics import recall_score, classification_report

y_pred = best_gb_model.predict(X_test)
test_recall = recall_score(y_test, y_pred)
print(f"\nRecall на тесте: {test_recall:.4f}")
print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred))



Поиск параметров для градиентного бустинга...
Fitting 5 folds for each of 18 candidates, totalling 90 fits

Лучшие параметры: {'gb__max_depth': 3, 'gb__min_samples_split': 5, 'gb__n_estimators': 200}
Лучший recall (CV): 0.9754

Recall на тесте: 0.9861

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.97      0.90      0.94        42
           1       0.95      0.99      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



Тестируем на лучшей модели

In [13]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

print("\n" + "="*50)
print("ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*50)

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ
Recall: 0.9861
Confusion Matrix:
[[38  4]
 [ 1 71]]


Используем те же параметры на своей модели

In [15]:
from boosting_classifier import GradientBoostingClassifier as gb


In [17]:
lr = gb(max_depth=3, n_estimators=200, min_samples_split=5)

In [18]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [19]:
print("\n" + "="*50)
print("ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*50)

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


ОЦЕНКА МОДЕЛИ sklearn НА ТЕСТОВОЙ ВЫБОРКЕ
Recall: 0.9306
Confusion Matrix:
[[38  4]
 [ 5 67]]


Загружаем данные

In [20]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

Делим на выборки

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Обучаем бейзлайн

In [23]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [25]:
dt = GradientBoostingRegressor()

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

In [26]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 2919.1342535376302
mae = 44.615906723273795
r2 = 0.4490278134530704


In [28]:
from boosting_regressor import GradientBoostingRegressor as gb

In [29]:
dt = gb()

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

In [30]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 2913.2973155144427
mae = 44.62579638490759
r2 = 0.4501295067038954


Работаем над данными

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
Подбираем параметры

In [32]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Создание пайплайна для градиентного бустинга
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingRegressor(random_state=42))
])

param_grid = {
    'gb__n_estimators': [100, 200],
    'gb__max_depth': [3, 5],
    'gb__min_samples_leaf': [1, 5]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

print("\nПоиск параметров для градиентного бустинга...")
grid_search.fit(X_train, y_train)

print(f"\nЛучшие параметры: {grid_search.best_params_}")
print(f"Лучший отрицательный MSE (CV): {grid_search.best_score_:.4f}")
print(f"Лучший MSE (CV): {-grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_

# Дополнительно: оценка на тестовой выборке
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print(f"\nРезультаты на тестовой выборке:")
print(f"MSE на тесте: {test_mse:.4f}")
print(f"MAE на тесте: {test_mae:.4f}")
print(f"R² на тесте: {test_r2:.4f}")


Поиск параметров для градиентного бустинга...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

Лучшие параметры: {'gb__max_depth': 3, 'gb__min_samples_leaf': 1, 'gb__n_estimators': 100}
Лучший отрицательный MSE (CV): -3800.0624
Лучший MSE (CV): 3800.0624

Результаты на тестовой выборке:
MSE на тесте: 2898.1981
MAE на тесте: 44.5930
R² на тесте: 0.4530


Предсказываем на лучшей модели

In [33]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [34]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 2898.1981350103324
mae = 44.5930196981176
r2 = 0.45297940252055224


Видим улучшение метрик


In [36]:
model = gb(max_depth=3, min_samples_leaf=1, n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [37]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"mse = {mse}")
print(f"mae = {mae}")
print(f"r2 = {r2}")

mse = 2914.7071672781403
mae = 44.71661863010624
r2 = 0.4498634041400924


Аналогично